In [ ]:
import pandas as pd
import csv
# Load the dataset
df = pd.read_csv('mxmh_survey_results.csv')




# Data Cleaning and Wrangling
Standardize Column Names

Converted all column names to lowercase and replaced spaces with underscores for easier access.

df.columns = df.columns.str.lower().str.replace(' ', '_')
print("Cleaned column names:")
print(df.columns)

# Handle Missing Values

music_effects: This is a key categorical variable. [cite_start] 
We will fill the missing values with "No effect" as a neutral default.
Other Categorical Columns: For other categorical columns like primary_streaming_service, while_working, etc., we will fill missing values with the mode (the most frequent value) as a reasonable assumption.
Numerical Columns (age, bpm): For numerical columns, we'll fill missing values with the median, which is less sensitive to outliers than the mean.

# Fill missing 'music_effects' with 'No effect'
df['music_effects'].fillna('No effect', inplace=True)

# Fill other categorical NaNs with the mode
for col in ['primary_streaming_service', 'while_working', 'instrumentalist', 'composer']:
mode_value = df[col].mode()[0]
df[col].fillna(mode_value, inplace=True)

# Fill numerical NaNs with the median
df['age'].fillna(df['age'].median(), inplace=True)
df['bpm'].fillna(df['bpm'].median(), inplace=True)

# Verify that most missing values are handled
print(df.isnull().sum())

# Correct Erroneous Data
During inspection of numerical data, an extremely high BPM value was noticed. This is likely a data entry error. Let's find and correct it.
A normal human heart rate, even during intense exercise, rarely exceeds 220 BPM, so we'll cap our BPM values at a reasonable upper limit (e.g., 250).

# Display descriptive statistics to spot outliers
print(df['bpm'].describe())
The max BPM is 999999999.0. This will replace any BPM over 250 with the median BPM.

# Find the median BPM to use for replacement
median_bpm = df['bpm'].median()

# Replace impossibly high BPM values
df['bpm'] = df['bpm'].apply(lambda x: median_bpm if x > 250 else x)

# Verify the correction
print("\nBPM statistics after correction:")
print(df['bpm'].describe())


# Final Data Check
print("Data cleaning complete. Here's the info for the cleaned DataFrame:")
df.info()


In [ ]:
Exploratory Data Analysis (EDA)

mental_health_cols = ['anxiety', 'depression', 'insomnia', 'ocd']

# Group by favorite genre and calculate mean scores
genre_mental_health = df.groupby('fav_genre')[mental_health_cols].mean().sort_values(by='anxiety', ascending=False)

genre_mental_health


Correlation Analysis

# Select numerical columns for correlation matrix
numerical_cols = ['age', 'hours_per_day', 'bpm'] + mental_health_cols

# Calculate the correlation matrix
corr_matrix = df[numerical_cols].corr()

# Plot a heatmap to visualize the correlations
plt.figure(figsize=(10, 7))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Matrix of Numerical Features')
plt.show()


Finding: The correlation matrix shows fairly weak relationships between most variables. Anxiety, depression, insomnia, and OCD scores are moderately correlated with each other, which is expected as these conditions often coexist. There appears to be a very weak positive correlation between hours_per_day and the mental health metrics, but it's not strong enough to be conclusive from this chart alone.


In [ ]:
Visualizations

Visualization 1: Average Anxiety by Favorite Music Genre

Story: I want to see if people who prefer certain music genres report higher levels of anxiety on average. This helps explore the stereotype that certain genres, like Metal or Rock, are associated with negative emotions, while others, like Classical, are associated with calmness.

Chart Justification: A bar chart is the perfect choice here because it's excellent for comparing a numerical value (average anxiety) across different categories (music genres). I sorted the bars in descending order to make it immediately clear which genres have the highest and lowest associated anxiety scores. I used a single color (skyblue) to keep the focus on the data itself rather than distracting with a complex color scheme.


# Prepare data for plotting
genre_anxiety = df.groupby('fav_genre')['anxiety'].mean().sort_values(ascending=False)

# Create the bar plot
plt.figure(figsize=(12, 8))
sns.barplot(x=genre_anxiety.values, y=genre_anxiety.index, palette='viridis')
plt.title('Average Anxiety Score by Favorite Music Genre', fontsize=16)
plt.xlabel('Average Anxiety Score (0-10)', fontsize=12)
plt.ylabel('Favorite Genre', fontsize=12)
plt.xticks(np.arange(0, 11, 1))
plt.grid(axis='x', linestyle='--', alpha=0.6)
plt.show()

Finding: The chart shows that fans of Rock and Metal music reported the highest average anxiety levels. Conversely, those who favor Lofi, Latin, and Country music reported the lowest levels. This doesn't imply causation but is an interesting pattern worth noting.

Visualization 2: Relationship Between Listening Hours and Depression

Story: Does listening to more music correlate with higher or lower self-reported depression? I want to investigate the pattern between the number of hours someone listens to music per day and their depression score.

Chart Justification: A scatter plot with a regression line (regplot) is ideal for visualizing the relationship between two continuous variables. Each point represents a survey respondent, and the line of best fit helps show the overall trend. A small amount of "jitter" was added to the data points to prevent overplotting where many respondents have the same values (e.g., 2 hours, depression score of 7). The blue color is standard and provides good contrast, while the semi-transparent points (scatter_kws={'alpha':0.4}) help reveal the density of the data.

# Create the regression plot
plt.figure(figsize=(10, 6))
sns.regplot(data=df, x='hours_per_day', y='depression',
x_jitter=0.5, y_jitter=0.5,
scatter_kws={'alpha': 0.3},
line_kws={'color': 'red'})
plt.title('Listening Hours per Day vs. Depression Score', fontsize=16)
plt.xlabel('Hours per Day', fontsize=12)
plt.ylabel('Depression Score (0-10)', fontsize=12)
plt.yticks(np.arange(0, 11, 1))
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()


Finding: The regression line has a slight positive slope, suggesting a weak positive correlation: as listening hours increase, the depression score tends to increase slightly. However, the points are widely dispersed, indicating that this relationship is not strong. There are many people who listen for many hours with low depression scores, etc. This tells us that listening time alone is not a strong predictor of depression.

Visualization 3: Distribution of Mental Health Scores

Story: What is the overall landscape of mental health in this survey's population? (Anxiety, Depression, Insomnia) to see their central tendencies and spread.

Chart Justification: Box plots are an excellent tool for comparing the distributions of several numerical variables at once. For each condition, the box plot clearly shows the median (the line inside the box), the interquartile range (the box itself, representing the middle 50% of data), and the spread of the data (the whiskers). This makes it easy to compare, whether the median anxiety score is higher than the median insomnia score and how their spreads differ. The pastel color palette is visually appealing and easy on the eyes.

# Reshape the data for easier plotting with seaborn
melted_df = df[mental_health_cols].melt(var_name='Condition', value_name='Score')

# Create the box plot
plt.figure(figsize=(12, 7))
sns.boxplot(data=melted_df, x='Condition', y='Score', palette='pastel')
plt.title('Distribution of Self-Reported Mental Health Scores', fontsize=16)
plt.xlabel('Mental Health Condition', fontsize=12)
plt.ylabel('Score (0-10)', fontsize=12)
plt.yticks(np.arange(0, 11, 1))
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()


Finding: The box plots reveal that the median scores for all conditions are relatively high (hovering between 4 and 6). Anxiety and Depression have very similar distributions, with their median scores being the highest among the four conditions. Insomnia scores are slightly lower and more spread out. OCD scores are the lowest on average, with a median around 2, and are more skewed towards the lower end of the scale.


# Key Findings
Data Quality: The initial dataset required significant cleaning, including standardizing column names, handling numerous missing values, and correcting outlier data points.

Genre & Anxiety: A clear pattern emerged where favorite genres like Rock and Metal were associated with higher average anxiety scores, while Lofi and Latin were associated with lower scores.

Listening Hours: There is a very weak positive correlation between the number of hours spent listening to music and self-reported scores for depression, anxiety, and insomnia. The weakness of this correlation suggests that listening time is not a major predictor of mental health status.